In [2]:
import torch
import torch.nn as nn
import numpy as np 
import pandas as pd

from pathlib import Path
from config import get_config, latest_weights_file_path
from train import get_model, get_ds, run_validation
from translate import translate_func
from validation import df1, df2
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
# Define the device
config = get_config()
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(config)
model = get_model(config, tokenizer_src.get_vocab_size(), tokenizer_tgt.get_vocab_size()).to(device)

# Load the pretrained weights
model_filename = latest_weights_file_path(config)
state = torch.load(model_filename)
model.load_state_dict(state['model_state_dict'])

Loading dataset from cache.
(80000, 1)
train and val sizes: 79920 80


<All keys matched successfully>

In [4]:
# run_validation(model, val_dataloader, tokenizer_src, tokenizer_tgt, config['seq_len'], device, lambda msg: print(msg), 0, None, num_examples=1)
# translate_func("Wir müssen allerdings vermeiden")

In [5]:
translated_sentences_dataset1 = list()
translated_sentences_dataset2 = list()

for ger_sentence, eng_sentence in zip(df1['German'], df1['English']):
    translated_sentences_dataset1.append([ger_sentence, translate_func(ger_sentence), eng_sentence])

for ger_sentence, eng_sentence in zip(df2['German'], df2['English']):
    translated_sentences_dataset2.append([ger_sentence, translate_func(ger_sentence), eng_sentence])

translated_sentences_dataset1 = np.array(translated_sentences_dataset1)
translated_sentences_dataset2 = np.array(translated_sentences_dataset2)

np.save('dataset1 result', translated_sentences_dataset1)
np.save('dataset2 result', translated_sentences_dataset2)

    SOURCE: Guten Morgen!
 PREDICTED: Tomorrow !  
------------------------------------------------------------------
    SOURCE: Wie geht es dir?
 PREDICTED: How can be transferred away .  
------------------------------------------------------------------
    SOURCE: Ich bin hungrig.
 PREDICTED: I am a farmer of people who have my  .  
------------------------------------------------------------------
    SOURCE: Entschuldigung, wo ist die Toilette?
 PREDICTED: For how is the file it is sense of research ?  
------------------------------------------------------------------
    SOURCE: Wie viel kostet das?
 PREDICTED: How much that is the costs ?  
------------------------------------------------------------------
    SOURCE: Ich spreche kein Deutsch.
 PREDICTED: I do not mention a German .  
------------------------------------------------------------------
    SOURCE: Was ist dein Name?
 PREDICTED: What is tantamount ?  
-------------------------------------------------------------

In [6]:
loaded_dataset1 = np.load("dataset1 result.npy")
loaded_dataset2 = np.load("dataset2 result.npy")

In [7]:
df = pd.DataFrame(loaded_dataset1)
df.columns =['German', 'Predicted English', 'True English']
df

,German,Predicted English,True English
0,Guten Morgen!,Tomorrow !,Good morning!
1,Wie geht es dir?,How can be transferred away .,How are you?
2,Ich bin hungrig.,I am a farmer of people who have my .,I am hungry.
3,"Entschuldigung, wo ist die Toilette?",For how is the file it is sense of research ?,"Excuse me, where is the restroom?"
4,Wie viel kostet das?,How much that is the costs ?,How much does that cost?
5,Ich spreche kein Deutsch.,I do not mention a German .,I don't speak German.
6,Was ist dein Name?,What is tantamount ?,What is your name?
7,Es tut mir leid.,I am sorry about that is about the Minutes .,I am sorry.
8,Woher kommst du?,Somalia is necessary .,Where are you from?
9,Ich liebe dich.,I love .,I love you.


In [8]:
df = pd.DataFrame(loaded_dataset2)
df.columns =['German', 'Predicted English', 'True English']
df

,German,Predicted English,True English
0,Die Sonne scheint am blauen Himmel über den Be...,The sun is still leaves as top of the blue .,The sun shines in the blue sky over the mounta...
1,Ich gehe mit meinem Hund im Park spazieren.,I am told before my dog .,I walk with my dog in the park.
2,Wir essen gerne Pizza und trinken kühles Bier.,We eat away from cheese beer .,We enjoy eating pizza and drinking cold beer.
3,Meine Schwester liest ein Buch in ihrem Zimmer.,My sister a book in her book .,My sister is reading a book in her room.
4,Der Zug fährt pünktlich zum Bahnhof und bringt...,The train currently travelling around the pass...,The train departs on time to the station and b...
5,Im Garten blühen bunte Blumen und grüne Sträuc...,At a tree houses of passionate desire and green .,"In the garden, colorful flowers and green bush..."
6,Er spielt Gitarre und singt schöne Lieder auf ...,He goes to show full evidence of evidence of e...,He plays the guitar and sings beautiful songs ...
7,Die Kinder spielen fröhlich im Park und lachen...,The children are regularly being driven out of...,The children play happily in the park and laug...
8,Der Kellner serviert köstliches Essen und erfr...,The file regularly took up hand in advance wor...,The waiter serves delicious food and refreshin...
9,Ich höre gerne klassische Musik und entspanne ...,I am delighted with my life more than happy .,I enjoy listening to classical music and relax...
